In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib as plt
import seaborn as sns

from itertools import product
from collections import Counter

In [2]:
from scripts.data_processing import GameResponses, Demographics

In [4]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account
cred = credentials.Certificate('E:/Behavioural cybersecurity/project-notes/cards-dd-game-61b9b7f26a6e.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

subjects_ref = db.collection(u'subjects')

In [5]:
uuids = [ # same players condition
    '611e4e160f41977f737035ef',
    '6112b4cdd72db936075c0db8',
    '611e28b12a630ff00cc467a8',
    '6108512871ec370e525ef4f0',
    '611e717148e6b271f0b70d6f',
    '5f415bea2ba0682e7fc98fbf',
    '611e6e0371738becff3551d8',
    '60ff0201788cef38c8695833',
    '6115e7dcd6cecc981dde3ed6', #incomplete response
    '611be41c1e9cca41ae48582a'
]

# uuids = [ # different players condition
#     '61005d2e9b8bac27eba5b984',
# '610995bd91de0e899effd73d',
# '599186630675b100014daddd',
# '6119884fd696463d717432c9',
# '610bb05be2e705f3e47e87fb',
# '610824959b488e57fa8a4a33',
# '60f0937892bd49086d6e2a77',
# '5cf656e90acc8f000147f643',
# '5fe292a28fd71dea34d04453',
# '5d341b57a48ae7000114152a'
# ]

### inspect main game responses

In [6]:
ttt = []

for uuid in uuids:
    ttt.append(subjects_ref.document(uuid).get())

In [7]:
condition = {}

for i, uuid in enumerate(uuids):
    cond = ttt[i].to_dict()['condition']
    condition[uuids[i]] = cond

In [8]:
# game task responses query reference
refs_block1 = []

for uuid in uuids:
    refs_block1.append(subjects_ref.document(uuid).collection(u'responses').document('game_block1').get())

In [9]:
# game task responses query reference
refs_block2 = []

for uuid in uuids:
    refs_block2.append(subjects_ref.document(uuid).collection(u'responses').document('game_block2').get())

In [10]:
# game task responses query reference
refs_block3 = []

for uuid in uuids:
    refs_block3.append(subjects_ref.document(uuid).collection(u'responses').document('game_block3').get())

In [11]:
# card values
transdict = {'blue': 1, 'red': -1}

### process main response data per user

In [12]:
honesty_ratings = []
RT_ratings = []

rand_pick_col = []
reported_col = []

for i in range(len(uuids)):
    block1_resp = GameResponses.from_dict(refs_block1[i].to_dict())
    block2_resp = GameResponses.from_dict(refs_block2[i].to_dict())
    block3_resp = GameResponses.from_dict(refs_block3[i].to_dict())
    
    i_honesty_ratings = [int(numeric_string) for numeric_string in block1_resp.honestyRating]
    i_honesty_ratings.extend([int(numeric_string) for numeric_string in block2_resp.honestyRating])
    i_honesty_ratings.extend([int(numeric_string) for numeric_string in block3_resp.honestyRating])
    honesty_ratings.append(i_honesty_ratings)
    
    i_pick = [transdict[x] for x in block1_resp.randomPickColour]
    i_pick.extend([transdict[x] for x in block2_resp.randomPickColour])
    i_pick.extend([transdict[x] for x in block3_resp.randomPickColour])
    rand_pick_col.append(i_pick)
    
    i_report = [x for x in block1_resp.reportColour]
    i_report.extend([x for x in block2_resp.reportColour])
    i_report.extend([x for x in block3_resp.reportColour])
    reported_col.append(i_report)
    
    i_RT_ratings = [int(numeric_string) for numeric_string in block1_resp.RThonesty]
    i_RT_ratings.extend([int(numeric_string) for numeric_string in block2_resp.RThonesty])
    i_RT_ratings.extend([int(numeric_string) for numeric_string in block3_resp.RThonesty])
    RT_ratings.append(i_RT_ratings)

In [13]:
# lie proportion per user
lie_prop = []
avg_honesty_rating = []

for i in range(len(uuids)):
    i_lie_prop = sum([x != y for x, y in zip(rand_pick_col[i], reported_col[i])])/90
    i_avg_rating = sum(honesty_ratings[i])/len(honesty_ratings[i])
    print(i_lie_prop, i_avg_rating)
    lie_prop.append(i_lie_prop)
    avg_honesty_rating.append(i_avg_rating)

0.4777777777777778 3.8666666666666667
0.6111111111111112 6.0
0.34444444444444444 3.533333333333333
0.03333333333333333 4.222222222222222
0.13333333333333333 2.6444444444444444
0.5666666666666667 3.0
0.7444444444444445 3.533333333333333
0.0 3.8333333333333335
0.4666666666666667 5.555555555555555
0.4444444444444444 2.6555555555555554


In [14]:
from scipy.stats import spearmanr, pearsonr

In [15]:
spearmanr(lie_prop, avg_honesty_rating)

SpearmanrResult(correlation=0.15197638592001614, pvalue=0.6751243317896831)

In [16]:
pearsonr(lie_prop, avg_honesty_rating)

(0.18814150306102617, 0.6027017338392772)

In [17]:
norm_honesty_ratings = []

for ratings in honesty_ratings:
    normed = [(v/6) for v in ratings]
    norm_honesty_ratings.append(normed)

In [18]:
import numexpr

norm_suspicion_ratings = numexpr.evaluate('(1 - norm_honesty_ratings)')

In [20]:
tmp = {uuids[i]: ratings for i, ratings in enumerate(norm_suspicion_ratings)}

# pd.DataFrame(tmp).to_json('final_pilot_diffplayer_normalized_suspicion_ratings.json')

In [21]:
tmp

{'611e4e160f41977f737035ef': array([0.5       , 0.        , 0.83333333, 0.        , 0.83333333,
        0.        , 0.        , 0.5       , 0.16666667, 0.        ,
        0.        , 0.5       , 0.        , 0.        , 0.16666667,
        0.        , 0.        , 0.83333333, 0.        , 0.16666667,
        0.16666667, 0.        , 0.16666667, 0.16666667, 0.        ,
        0.16666667, 0.33333333, 0.        , 0.83333333, 0.66666667,
        0.5       , 0.        , 0.16666667, 0.5       , 0.        ,
        0.        , 0.        , 0.16666667, 0.5       , 0.        ,
        0.33333333, 0.66666667, 0.        , 0.        , 0.16666667,
        0.66666667, 0.33333333, 0.        , 0.16666667, 0.        ,
        0.83333333, 0.        , 1.        , 0.33333333, 0.33333333,
        0.5       , 1.        , 0.33333333, 0.66666667, 0.        ,
        0.66666667, 0.83333333, 0.        , 0.5       , 0.        ,
        0.66666667, 0.33333333, 0.33333333, 0.33333333, 1.        ,
        0.33333333, 

In [195]:
tmp = {uuids[i]: ratings for i, ratings in enumerate(honesty_ratings)}

In [139]:
# inspect block order effects

def mean(x):
    return sum(x)/len(x)

In [196]:
avg_nolie_block_ratings = []
avg_lie33_block_ratings = []
avg_lie50_block_ratings = []

sd_nolie_block_ratings = []
sd_lie33_block_ratings = []
sd_lie50_block_ratings = []

for i, uuid in enumerate(uuids):
    if condition[uuid] == "11":
        nolie = tmp[uuid][:30]
        avg_nolie_block_ratings.append(mean(nolie))
        sd_nolie_block_ratings.append(np.std(nolie))
#         print(i, "mean honesty rating", mean(nolie))
#         print(i, "SD honesty rating", np.std(nolie))
        
        lie33 = tmp[uuid][30:60]
        avg_lie33_block_ratings.append(mean(lie33))
        sd_lie33_block_ratings.append(np.std(lie33))
        
        lie50 = tmp[uuid][60:]
        avg_lie50_block_ratings.append(mean(lie50))
        sd_lie50_block_ratings.append(np.std(lie50))
        
    if condition[uuid] == "12":
        nolie = tmp[uuid][:30]
        avg_nolie_block_ratings.append(mean(nolie))
        sd_nolie_block_ratings.append(np.std(nolie))
#         print(i, "mean honesty rating", mean(nolie))
#         print(i, "SD honesty rating", np.std(nolie))
        
        lie33 = tmp[uuid][60:]
        avg_lie33_block_ratings.append(mean(lie33))
        sd_lie33_block_ratings.append(np.std(lie33))
        
        lie50 = tmp[uuid][30:60]
        avg_lie50_block_ratings.append(mean(lie50))
        sd_lie50_block_ratings.append(np.std(lie50))
        
    if condition[uuid] == "21":
        nolie = tmp[uuid][30:60]
        avg_nolie_block_ratings.append(mean(nolie))
        sd_nolie_block_ratings.append(np.std(nolie))
#         print(i, "mean honesty rating", mean(nolie))
#         print(i, "SD honesty rating", np.std(nolie))
        
        lie33 = tmp[uuid][:30]
        avg_lie33_block_ratings.append(mean(lie33))
        sd_lie33_block_ratings.append(np.std(lie33))
        
        lie50 = tmp[uuid][60:]
        avg_lie50_block_ratings.append(mean(lie50))
        sd_lie50_block_ratings.append(np.std(lie50))
        
    if condition[uuid] == "22":
        nolie = tmp[uuid][60:]
        avg_nolie_block_ratings.append(mean(nolie))
        sd_nolie_block_ratings.append(np.std(nolie))
#         print(i, "mean honesty rating", mean(nolie))
#         print(i, "SD honesty rating", np.std(nolie))
        
        lie33 = tmp[uuid][:30]
        avg_lie33_block_ratings.append(mean(lie33))
        sd_lie33_block_ratings.append(np.std(lie33))
        
        lie50 = tmp[uuid][30:60]
        avg_lie50_block_ratings.append(mean(lie50))
        sd_lie50_block_ratings.append(np.std(lie50))
        
    if condition[uuid] == "31":
        nolie = tmp[uuid][30:60]
        avg_nolie_block_ratings.append(mean(nolie))
        sd_nolie_block_ratings.append(np.std(nolie))
#         print(i, "mean honesty rating", mean(nolie))
#         print(i, "SD honesty rating", np.std(nolie))
        
        lie33 = tmp[uuid][60:]
        avg_lie33_block_ratings.append(mean(lie33))
        sd_lie33_block_ratings.append(np.std(lie33))
        
        lie50 = tmp[uuid][:30]
        avg_lie50_block_ratings.append(mean(lie50))
        sd_lie50_block_ratings.append(np.std(lie50))
    if condition[uuid] == "32":
        nolie = tmp[uuid][60:]
        avg_nolie_block_ratings.append(mean(nolie))
        sd_nolie_block_ratings.append(np.std(nolie))
#         print(i, "mean honesty rating", mean(nolie))
#         print(i, "SD honesty rating", np.std(nolie))
                
        lie33 = tmp[uuid][30:60]
        avg_lie33_block_ratings.append(mean(lie33))
        sd_lie33_block_ratings.append(np.std(lie33))
        
        lie50 = tmp[uuid][:30]
        avg_lie50_block_ratings.append(mean(lie50))
        sd_lie50_block_ratings.append(np.std(lie50))
    else:
        pass

In [197]:
block_lieorder_avg_ratings = list(zip(avg_nolie_block_ratings, avg_lie33_block_ratings, avg_lie50_block_ratings))

In [199]:
block_lieorder_sd_ratings = list(zip(sd_nolie_block_ratings, sd_lie33_block_ratings, sd_lie50_block_ratings))

In [201]:
from scipy.stats import chisquare

In [202]:
chisquare(avg_nolie_block_ratings)

Power_divergenceResult(statistic=3.879077099455432, pvalue=0.9191894604148341)

In [203]:
chisquare(avg_lie33_block_ratings)

Power_divergenceResult(statistic=5.1985923929320155, pvalue=0.8166641465375826)

In [204]:
chisquare(avg_lie50_block_ratings)

Power_divergenceResult(statistic=3.2765027322404356, pvalue=0.9523166295273383)

In [210]:
avg_nolie_block_ratings

[4.166666666666667,
 6.0,
 4.266666666666667,
 3.4,
 2.4,
 3.2333333333333334,
 2.5,
 4.266666666666667,
 5.866666666666666,
 2.6666666666666665]

In [215]:
condition

{'611e4e160f41977f737035ef': '21',
 '6112b4cdd72db936075c0db8': '21',
 '611e28b12a630ff00cc467a8': '11',
 '6108512871ec370e525ef4f0': '32',
 '611e717148e6b271f0b70d6f': '31',
 '5f415bea2ba0682e7fc98fbf': '31',
 '611e6e0371738becff3551d8': '22',
 '60ff0201788cef38c8695833': '12',
 '6115e7dcd6cecc981dde3ed6': '12',
 '611be41c1e9cca41ae48582a': '11'}

In [17]:
#     i_report_honesty = np.array([transdict[x] for x in block1_resp.randomPickColour]) != block1_resp.reportColour
#     i_report_honesty.extend(np.array([transdict[x] for x in block2_resp.randomPickColour]) != block2_resp.reportColour)
#     i_report_honesty.extend(np.array([transdict[x] for x in block3_resp.randomPickColour]) != block3_resp.reportColour)
#     if_lied.append(i_report_honesty)

### inspect demographics data

In [18]:
# demographics survey query reference
refs_demographics = []
for uuid in uuids:
    refs_demographics.append(subjects_ref.document(uuid).collection(u'responses').document('demographics').get())

In [61]:
age = []

for i, feedback in enumerate(refs_demographics):
    if feedback.to_dict() is None:
        continue
    print(i, feedback.to_dict()['age'])
    age.append(int(feedback.to_dict()['age']))

0 19
1 18
2 18
3 30
4 20
5 19
6 18
7 23
8 31


In [65]:
sum(age)/len(age)

21.77777777777778

In [21]:
gender = []

for i, feedback in enumerate(refs_demographics):
    print(i, feedback.to_dict()['gender'])
    gender.append(feedback.to_dict()['gender'])

0 male
1 female
2 male
3 female
4 female
5 male
6 female
7 female
8 female


In [22]:
spearmanr(age, avg_honesty_rating[:len(gender)])

SpearmanrResult(correlation=-0.2393512117319333, pvalue=0.5350802499319773)

### inspect EQ scores

In [23]:
# EQ survey query reference
refs_eq = []

for uuid in uuids:
    refs_eq.append(subjects_ref.document(uuid).collection(u'responses').document('EQ').get())

In [24]:
eq_items_r = [
    "I find it hard to know what to do in a social situation",
    "I often find it difficult to judge if something is rude or polite",
    "In a conversation I tend to focus on my own thoughts rather than on what my listener might be thinking",
    "It is hard for me to see why some things upset people so much", 
    "I can’t always see why someone should have felt offended by a remark",
    "Other people often say that I am insensitive though I don’t always see why"
]

eq_items = ["I can easily tell if someone else wants to enter a conversation",
            "I really enjoy caring for other people",
            "I can pick up quickly if someone says one thing but means another",
            "I find it easy to put myself in somebody else’s shoes",
            "I am good at predicting how someone will feel",
            "I am quick to spot when someone in a group is feeling awkward or uncomfortable",
            "I don’t tend to find social situations confusing",
            "Other people tell me I am good at understanding how they are feeling and what they are thinking",
            "I can easily tell if someone else is interested or bored with what I am saying",
            "Friends usually talk to me about their problems as they say that I am very understanding",
            "I can sense if I am intruding even if the other person doesn’t tell me",
            "I can tune into how someone else feels rapidly and intuitively",
            "I can easily work out what another person might want to talk about",
            "I can tell if someone is masking their true emotion",
            "I am good at predicting what someone will do",
            "I tend to get emotionally involved with a friend’s problem"
           ]

eq_catch = [
    "Please select the middle answer option",
    "Please select the first answer option",
    "Please select the last answer option"
]

In [32]:
eq_responses = []

for uuid in uuids:
    for i, resp in enumerate(refs_eq):
        i_eq = []
        if resp.to_dict() is None:
            continue
        for it in eq_items:
            i_eq.append(resp.to_dict()[it])
        for jt in eq_items_r:
            ar = resp.to_dict()[jt]
            ar = numexpr.evaluate(f'(7 - ar)')
            i_eq.append(ar.item())
        eq_responses.append(i_eq)

In [33]:
len(eq_responses)

81

In [38]:
eq_responses = eq_responses[:len(uuids)]

In [39]:
eq_responses

[[6, 7, 6, 5, 4, 5, 6, 6, 5, 5, 4, 5, 4, 4, 4, 5, 5, 5, 4, 5, 4, 5],
 [5, 7, 6, 7, 3, 7, 2, 7, 6, 7, 7, 5, 5, 4, 2, 7, 2, 6, 6, 2, 4, 6],
 [1, 3, 6, 6, 7, 3, 6, 7, 6, 7, 7, 6, 6, 7, 6, 7, 2, 1, 2, 1, 1, 2],
 [5, 7, 5, 1, 6, 6, 3, 5, 6, 6, 5, 7, 6, 4, 5, 5, 1, 5, 0, 6, 5, 6],
 [5, 7, 6, 7, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 5, 5, 5, 3, 6, 4, 5],
 [6, 5, 6, 6, 5, 7, 6, 7, 5, 6, 6, 6, 7, 5, 6, 5, 4, 2, 0, 0, 2, 5],
 [5, 6, 6, 2, 5, 6, 6, 5, 5, 6, 6, 5, 5, 6, 6, 1, 2, 5, 4, 0, 5, 6],
 [6, 6, 6, 6, 6, 6, 2, 5, 6, 5, 7, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 5],
 [5, 5, 6, 5, 5, 5, 6, 6, 6, 5, 5, 4, 4, 4, 5, 6, 4, 4, 3, 5, 2, 2]]

In [40]:
avg_eq_nr = [sum(r)/len(r) for r in eq_responses]

In [97]:
spearmanr(avg_eq_nr, lie_prop)

SpearmanrResult(correlation=-0.11666666666666665, pvalue=0.7650079429261462)

In [98]:
pearsonr(avg_eq_nr, lie_prop)

(-0.2707677861637728, 0.48099509093735415)

In [44]:
eq_catch_resp = []

for uuid in uuids:
    for i, resp in enumerate(refs_eq):
        if resp.to_dict() is None:
            continue
        i_eq = []
        for it in eq_catch:
            i_eq.append(resp.to_dict()[it])
        eq_catch_resp.append(i_eq)

In [45]:
# check if people succeeded at all catch questions

eq_catch_resp[:len(uuids)]

[[4, 1, 7],
 [4, 1, 7],
 [4, 1, 7],
 [4, 1, 7],
 [4, 1, 7],
 [4, 1, 7],
 [4, 1, 7],
 [4, 1, 7],
 [4, 1, 7]]

### RTs

In [86]:
for i, uuid in enumerate(uuids):
    print(spearmanr(RT_ratings[i], honesty_ratings[i]))

SpearmanrResult(correlation=0.08944468357523604, pvalue=0.4018242969895841)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=0.18044191959144018, pvalue=0.08877529822936306)
SpearmanrResult(correlation=-0.19214922669616544, pvalue=0.06962335284518002)
SpearmanrResult(correlation=0.3156197423081183, pvalue=0.0024436527754634987)
SpearmanrResult(correlation=0.2494472467280193, pvalue=0.01774022871460874)
SpearmanrResult(correlation=-0.017123192210922695, pvalue=0.8727346692674682)
SpearmanrResult(correlation=0.027234420416241203, pvalue=0.7988744405488555)
SpearmanrResult(correlation=-0.10767294038839022, pvalue=0.3124297259978243)


In [92]:
avg_RT_ratings = [sum(rt)/len(rt) for rt in RT_ratings]
spearmanr(avg_RT_ratings, lie_prop)

SpearmanrResult(correlation=0.2333333333333333, pvalue=0.5456987782182727)

### inspect feedback and debrief question responses

In [46]:
refs_feedback = []

for uuid in uuids:
    refs_feedback.append(subjects_ref.document(uuid).collection(u'responses').document('feedback').get())

In [47]:
for i, feedback in enumerate(refs_feedback):
    print(uuids[i], feedback.to_dict()['studyPurpose'])

611e4e160f41977f737035ef To understand people's ability to perceive dishonesty and how honest they are
6112b4cdd72db936075c0db8 Your guessing skills.
611e28b12a630ff00cc467a8 To see how honest people are with each other despite the fact they don't know one another. 
6108512871ec370e525ef4f0 To compare honesty with comprehension!
611e717148e6b271f0b70d6f seeing peoples honesty and greed
5f415bea2ba0682e7fc98fbf I think the purpose of the study was to see honesty in players, instead of testing a game. I think you're looking for people who can easily notice a bug or glaring mistake in the rules and use it to their advantage, whilst also looking to see if people will be truly honest. You're also trying to measure if people can notice deception.
611e6e0371738becff3551d8 To see whether people feel too guilty to lie about the colour of the card that the computer has selected. There is absolutely no incentive to report red: you will either gain nothing or lose five pence.
60ff0201788cef38c8695

In [48]:
for i, feedback in enumerate(refs_feedback):
    print(uuids[i], feedback.to_dict()['understoodTask'])

611e4e160f41977f737035ef 6
6112b4cdd72db936075c0db8 5
611e28b12a630ff00cc467a8 6
6108512871ec370e525ef4f0 5
611e717148e6b271f0b70d6f 6
5f415bea2ba0682e7fc98fbf 6
611e6e0371738becff3551d8 5
60ff0201788cef38c8695833 6
611be41c1e9cca41ae48582a 4


In [49]:
for i, feedback in enumerate(refs_feedback):
    print(uuids[i], feedback.to_dict()['concerns'])

611e4e160f41977f737035ef The money incentive in winning points by picking blue, although i suppose that is necessary to create dishonesty for the study.
6112b4cdd72db936075c0db8 No
611e28b12a630ff00cc467a8 Not at all
6108512871ec370e525ef4f0 I didn't realize in the first game that there were questions telling me to pick a certain answer on the scale. I though I just didn't select the answer when I hit submit. Therefore some of those answers may not be correct.
611e717148e6b271f0b70d6f no
5f415bea2ba0682e7fc98fbf It's broken. You can just keep selecting blue over and over again to get points and prevent the other player from getting any, unless they also pick blue every time and make it a tie each round.
611e6e0371738becff3551d8 I was concerned that by picking blue all the time I was being selfish and not giving other players the opportunity to pick up bonuses by winning games.
60ff0201788cef38c8695833 No
611be41c1e9cca41ae48582a no


In [50]:
for i, feedback in enumerate(refs_feedback):
    print(uuids[i], feedback.to_dict()['yourGoal'])

611e4e160f41977f737035ef The bonus money was a strong incentive, although it felt wrong constantly abusing it if it seemed the other player was being honest
6112b4cdd72db936075c0db8 To win bonus points.
611e28b12a630ff00cc467a8 My goal was to answer honestly. However, eventually the opposing person selected the blue card a suspicious amount of times making me do the same.
6108512871ec370e525ef4f0 My goal was to be honest when selecting my card picks though there were a few instances where I missed which card the computer selected due to environmental distractions. 
611e717148e6b271f0b70d6f being honest 
5f415bea2ba0682e7fc98fbf To get as many points as possible, beat the other player. I felt bad for being so dishonest though.
611e6e0371738becff3551d8 To get as many bonuses as possible
60ff0201788cef38c8695833 To click the computer's randomized card choice
611be41c1e9cca41ae48582a to try and work out what the other player would do


In [51]:
for i, feedback in enumerate(refs_feedback):
    print(uuids[i], feedback.to_dict()['othersGoal'])

611e4e160f41977f737035ef The first 2 appeared quite honest while the third seemed to be incentivised by the money to be dishonest.
6112b4cdd72db936075c0db8 To make me lose.
611e28b12a630ff00cc467a8 I believe the first 2 games the other play answered honestly but by the third i assumed that their goal was to win as much as possible.
6108512871ec370e525ef4f0 I believe the first player as playing honestly but the more I played, the more I felt the person was trying to in the most money by selecting the blue card frequently.
611e717148e6b271f0b70d6f earning money 
5f415bea2ba0682e7fc98fbf The first two players I believe didn't get that there was a glaring mistake in the rules of the game, as they kept picking the red card over and over again, giving me points each turn. However, the third player I believe started to catch onto it, and was picking blue over and over again, even when there was only 1 blue card on the row. I respect the third player.

There's a possibility that you guys might

In [52]:
for i, feedback in enumerate(refs_feedback):
    print(uuids[i], feedback.to_dict()['goodDetection'])

611e4e160f41977f737035ef 5
6112b4cdd72db936075c0db8 6
611e28b12a630ff00cc467a8 5
6108512871ec370e525ef4f0 4
611e717148e6b271f0b70d6f 6
5f415bea2ba0682e7fc98fbf 5
611e6e0371738becff3551d8 4
60ff0201788cef38c8695833 5
611be41c1e9cca41ae48582a 3


In [53]:
for i, feedback in enumerate(refs_feedback):
    print(uuids[i], feedback.to_dict()['howHonest'])

611e4e160f41977f737035ef 1
6112b4cdd72db936075c0db8 6
611e28b12a630ff00cc467a8 3
6108512871ec370e525ef4f0 5
611e717148e6b271f0b70d6f 5
5f415bea2ba0682e7fc98fbf 0
611e6e0371738becff3551d8 0
60ff0201788cef38c8695833 6
611be41c1e9cca41ae48582a 1


In [54]:
for i, feedback in enumerate(refs_feedback):
    print(uuids[i], feedback.to_dict()['howMany'])

611e4e160f41977f737035ef 3
6112b4cdd72db936075c0db8 3
611e28b12a630ff00cc467a8 3
6108512871ec370e525ef4f0 3
611e717148e6b271f0b70d6f 9
5f415bea2ba0682e7fc98fbf 1
611e6e0371738becff3551d8 3
60ff0201788cef38c8695833 3
611be41c1e9cca41ae48582a 3


In [55]:
for i, feedback in enumerate(refs_feedback):
    print(uuids[i], feedback.to_dict()['playAgain'])

611e4e160f41977f737035ef 1
6112b4cdd72db936075c0db8 1
611e28b12a630ff00cc467a8 1
6108512871ec370e525ef4f0 1
611e717148e6b271f0b70d6f 0
5f415bea2ba0682e7fc98fbf 0
611e6e0371738becff3551d8 1
60ff0201788cef38c8695833 1
611be41c1e9cca41ae48582a 1
